# AI in Civic Tech: Public Feedback Analysis
Citizens provide feedback on bike lanes, roadworks, and parks.

This notebook explores using AI to:
- Extract sentiment
- Flag follow-up
- Output structured data

In [ ]:
# Install Dependencies
!pip install transformers textblob --quiet
from transformers import pipeline
from textblob import TextBlob
import json

In [1]:
# Samples
sample_comments = [
    "The new bike lanes on King Street are great, but the signs are confusing for drivers.",
    "Park upgrade is nice, but the lighting at night is still terrible and unsafe.",
    "I appreciate the smoother roads after the construction—finally no potholes!"
]

In [ ]:
# Hugging Face Sentiment Classification
classifier = pipeline("sentiment-analysis")

def classify_with_huggingface(text):
    result = classifier(text)[0]
    sentiment = result['label'].upper()
    followup = "Y" if sentiment == "NEGATIVE" else "N"
    reason = "Safety or dissatisfaction reported." if followup == "Y" else "No urgent concern."
    return {
        "features": [{
            "feature_name": "infrastructure",
            "sentiment": sentiment,
            "followup": followup,
            "followup_reason": reason
        }]
    }

for comment in sample_comments:
    print(json.dumps(classify_with_huggingface(comment), indent=2))

In [ ]:
# TextBlob Rule Based Baseline
def classify_with_textblob(text):
    polarity = TextBlob(text).sentiment.polarity
    sentiment = "NEGATIVE" if polarity < -0.1 else "POSITIVE" if polarity > 0.1 else "NEUTRAL"
    followup = "Y" if sentiment == "NEGATIVE" else "N"
    reason = "Negative polarity detected." if followup == "Y" else "No concern."
    return {
        "features": [{
            "feature_name": "infrastructure",
            "sentiment": sentiment,
            "followup": followup,
            "followup_reason": reason
        }]
    }

for comment in sample_comments:
    print(json.dumps(classify_with_textblob(comment), indent=2))

## Gemini LLM:
In this section, we use the Gemini LLM to perform sentiment analysis and identify whether a follow-up action is needed based on civic feedback.

This demonstrates how a general-purpose LLM can replicate and enrich sentiment classification tasks.

In [2]:
from google import genai

client = genai.Client(api_key="<ADD YOUR API KEY HERE>")

In [7]:
# Use Gemini to analyze feedback with corrected JSON template formatting
def analyse_with_gemini(text):
    prompt = f"""
You are an assistant helping a city council analyze citizen feedback.
Given this comment:
'{text}'

Please classify the sentiment as POSITIVE, NEGATIVE, or NEUTRAL.
Also, indicate if follow-up is required (Y or N), and give a brief reason.

Respond strictly in JSON format like this:
{{
  "sentiment": "POSITIVE",
  "followup": "N",
  "reason": "No concern reported."
}}
"""
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash", contents=prompt
        )
        return response.text.strip()
    except Exception as e:
        return str(e)

In [8]:
# 📊 Apply Gemini classification to each comment
# This demo just shows the first 3 results for quick testing
for comment in sample_comments[:3]:
    print(f"Feedback: {comment}")
    print(analyse_with_gemini(comment))

Feedback: The new bike lanes on King Street are great, but the signs are confusing for drivers.
```json
{
  "sentiment": "POSITIVE",
  "followup": "Y",
  "reason": "While the overall sentiment towards the bike lanes is positive, there is a specific concern about confusing signage that warrants further investigation and potential action."
}
```
Feedback: Park upgrade is nice, but the lighting at night is still terrible and unsafe.
```json
{
  "sentiment": "NEGATIVE",
  "followup": "Y",
  "reason": "Reports unsafe lighting conditions at night."
}
```
Feedback: I appreciate the smoother roads after the construction—finally no potholes!
```json
{
  "sentiment": "POSITIVE",
  "followup": "N",
  "reason": "No concern reported."
}
```


## 📘 Reflection
- Where did the models agree/disagree?
- Which model handled nuance better?